In [1]:
# Cell 1: load & pad
import tifffile, numpy as np, os, re
from glob import glob

# Input folder
input_dir = r'D:\cjDove-DAE8-Systemic'

# Gather & sort by Section→Plate→Row→Col
def sort_key(path):
    fn = os.path.basename(path)
    m = re.match(r'.*P(\d+)-([A-C])(\d+)_S(\d+)', fn)
    if not m:
        return fn
    plate, rowL, col, sec = m.groups()
    return (int(sec), int(plate), ord(rowL)-ord('A'), int(col))

tiff_paths = sorted(glob(os.path.join(input_dir, '*.tif')), key=sort_key)
print(f"🔍 Found {len(tiff_paths)} TIFFs")

# Find max dims
all_imgs = []
max_h = max_w = 0
for p in tiff_paths:
    img = tifffile.imread(p)  # shape either (4,H,W) or (H,W,4)
    if img.ndim==3 and img.shape[2]==4:
        img = np.moveaxis(img, 2, 0)
    assert img.shape[0]==4, "Expected 4 channels"
    _, h, w = img.shape
    max_h, max_w = max(max_h,h), max(max_w,w)
    all_imgs.append(img)

print(f"✅ Loaded, max size = ({max_h},{max_w})")

# Center‑pad to (4, max_h, max_w)
padded = []
for img in all_imgs:
    C, h, w = img.shape
    pad = np.zeros((4, max_h, max_w), dtype=img.dtype)
    y0 = (max_h-h)//2
    x0 = (max_w-w)//2
    pad[:, y0:y0+h, x0:x0+w] = img
    padded.append(pad)

# Stack into volume: (C, N_sections, H, W)
volume = np.stack(padded, axis=1)
print("✅ Padded all images → volume.shape =", volume.shape)


🔍 Found 14 TIFFs
✅ Loaded, max size = (4934,5896)
✅ Padded all images → volume.shape = (4, 14, 4934, 5896)


In [2]:
# Cell 2: write CloudVolume per section/channel
import os
import numpy as np
from cloudvolume import CloudVolume

# Parameters
output_base = r'D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section'
channel_names = ['Nissl','aTH','dTom','aRFP']
chunk_size = [64, 64, 1]
resolution = [1, 1, 1]

C, N, H, W = volume.shape
print(f"Writing {N} sections × {C} channels, each {W}×{H}×1")

for idx in range(N):
    for ch in range(C):
        img = volume[ch, idx]            # (H, W)
        # reorder to (X, Y, Z=1, C=1)
        img_xyzc = img.transpose(1,0)[..., np.newaxis, np.newaxis]

        # Build output folder
        sec_dir = os.path.join(output_base, f'section_{idx:02d}', channel_names[ch])
        os.makedirs(sec_dir, exist_ok=True)
        uri = f'file:///{sec_dir}'.replace('\\','/')

        # Create metadata
        info = CloudVolume.create_new_info(
            num_channels=1,
            layer_type='image',
            data_type=str(img.dtype),
            encoding='raw',
            resolution=resolution,
            voxel_offset=[0,0,0],
            volume_size=[W, H, 1],
            chunk_size=chunk_size,
        )

        # Write
        vol = CloudVolume(uri, info=info, compress=False)
        vol.commit_info()
        vol[0:W, 0:H, 0:1, 0:1] = img_xyzc

        print(f"✅ Sec {idx:02d}, {channel_names[ch]} → {sec_dir}")

print("🎉 Done writing all sections/channels.")


Writing 14 sections × 4 channels, each 5896×4934×1


Uploading: 100%|██████████| 7254/7254 [00:04<00:00, 1517.05it/s]


✅ Sec 00, Nissl → D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\section_00\Nissl


Uploading: 100%|██████████| 7254/7254 [00:04<00:00, 1484.33it/s]


✅ Sec 00, aTH → D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\section_00\aTH


...

✅ Sec 13, aTH → D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\section_13\aTH


Uploading: 100%|██████████| 7254/7254 [00:04<00:00, 1485.55it/s]


✅ Sec 13, dTom → D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\section_13\dTom


Uploading: 100%|██████████| 7254/7254 [00:04<00:00, 1496.97it/s]

✅ Sec 13, aRFP → D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\section_13\aRFP
🎉 Done writing all sections/channels.


In [3]:
import os
import numpy as np
from cloudvolume import CloudVolume

base_dir = r'D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section'
output_dir = os.path.join(base_dir, "stitched_composite")
os.makedirs(output_dir, exist_ok=True)

channel_names = ["Nissl", "aTH", "dTom", "aRFP"]

# Detect all sections automatically
sections = sorted(
    d for d in os.listdir(base_dir)
    if d.startswith("section_") and os.path.isdir(os.path.join(base_dir, d))
    and not d.endswith("_4ch") and not d.endswith("composite")
)

print(f"Found {len(sections)} sections: {sections}")

# Collect all section arrays
all_sections = []
for i, section in enumerate(sections):
    ch_imgs = []
    print(f"\n=== Section {section} ({i+1}/{len(sections)}) ===")
    for ch in channel_names:
        path = os.path.join(base_dir, section, ch)
        print(f"  Loading {ch} from {path} ...", end=' ')
        try:
            vol = CloudVolume(f'file:///{path}', bounded=False, progress=False)
            arr = vol[:, :, :].squeeze()
            ch_imgs.append(arr)
            print("OK")
        except Exception as e:
            print(f"ERROR: {e}")
            break
    if len(ch_imgs) != 4:
        print(f"  Skipping {section} (not all channels loaded)")
        continue
    # Stack channels (H, W, 4)
    section_stack = np.stack(ch_imgs, axis=-1)
    all_sections.append(section_stack)
    print(f"  Section shape: {section_stack.shape}")

# Now stack all sections into (H, W, N_sections, 4)
if len(all_sections) == 0:
    print("❌ No sections loaded. Exiting.")
else:
    # Make sure all sections are the same shape (pad if needed)
    H, W, _ = all_sections[0].shape
    for idx, s in enumerate(all_sections):
        if s.shape[0] != H or s.shape[1] != W:
            raise ValueError(f"Section {idx} shape mismatch: {s.shape} (expected {(H,W,4)})")

    volume = np.stack(all_sections, axis=2)  # (H, W, Z, 4)
    print(f"\n✅ Final composite volume shape: {volume.shape}")

    # CloudVolume expects (X, Y, Z, C) for 4 channels, but volume_size omits the channel dimension!
    info = CloudVolume.create_new_info(
        num_channels=4,
        layer_type='image',
        data_type='uint16',
        encoding='raw',
        resolution=[1,1,1],
        voxel_offset=[0,0,0],
        chunk_size=[64,64,1],
        volume_size=list(volume.shape[:3]),  # [X, Y, Z]
    )

    vol_out = CloudVolume("file:///" + output_dir, info=info, compress=False, progress=True)
    vol_out.commit_info()
    print("📝 Writing to CloudVolume (this may take a while)...")
    vol_out[:, :, :] = volume  # <--- FIX: do not include channel dimension here!
    print(f"🎉 Finished writing 4-channel stitched composite at {output_dir}")


Found 14 sections: ['section_00', 'section_01', 'section_02', 'section_03', 'section_04', 'section_05', 'section_06', 'section_07', 'section_08', 'section_09', 'section_10', 'section_11', 'section_12', 'section_13']

=== Section section_00 (1/14) ===
  Loading Nissl from D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\section_00\Nissl ... OK
  Loading aTH from D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\section_00\aTH ... OK
  Loading dTom from D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\section_00\dTom ... OK
  Loading aRFP from D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\section_00\aRFP ... OK
  Section shape: (5896, 4934, 4)

=== Section section_01 (2/14) ===
  Loading Nissl from D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\section_01\Nissl ... OK
  Loading aTH from D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\section_01\aTH ... OK
  Loading dTom from D:\cjDove-DAE

Uploading: 100%|██████████| 101556/101556 [01:18<00:00, 1294.79it/s]

🎉 Finished writing 4-channel stitched composite at D:\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\stitched_composite


In [4]:
# Build one stacked precomputed layer per channel from per-section layers on D:
# Base dir MUST contain subfolders like:
#   D:\...\confocal_neuroglancer_dataset_per_section\section_001\DAPI\info
#   D:\...\confocal_neuroglancer_dataset_per_section\section_001\GFP\info
#   ... (TXRED, CY5)
#
# Output layers will be written alongside base_dir as:
#   D:\...\confocal_neuroglancer_dataset_per_section\DAPI_split\info
#   ... GFP_split, TXRED_split, CY5_split

import re
import numpy as np
from pathlib import Path
from cloudvolume import CloudVolume

# ---- CONFIG ----
base_dir = Path(r"C:\D\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section")
channels = ["Nissl", "aTH", "dTom", "aRFP"]
chunk_xy = 128  # adjust if you like
dtype    = "uint16"

def cloudpath(p: Path) -> str:
    """Windows-safe 'file://D:/...' (no triple slash)."""
    return "file://" + p.resolve().as_posix()

# ---- Discover sections in numeric order ----
sections = sorted(
    (p for p in base_dir.iterdir() if p.is_dir() and p.name.lower().startswith("section_")),
    key=lambda p: int(re.findall(r'\d+', p.name)[-1]) if re.findall(r'\d+', p.name) else p.name
)
print(f"Found {len(sections)} sections:", [s.name for s in sections[:5]], "..." if len(sections) > 5 else "")

if not sections:
    raise SystemExit(f"No 'section_*' folders found in {base_dir}")

# ---- Sanity: every section must have each channel with an 'info' ----
for ch in channels:
    missing = [s.name for s in sections if not (s/ch/'info').exists()]
    if missing:
        raise SystemExit(f"Missing precomputed 'info' for channel {ch} in sections (first 5 shown): {missing[:5]}")

# ---- Determine max X/Y across sections (use DAPI scale 0) ----
sizes = []
for s in sections:
    vol = CloudVolume(cloudpath(s / "DAPI"), progress=False, bounded=False)
    sx, sy, sz = map(int, vol.info["scales"][vol.mip]["size"])  # [X,Y,Z], typically Z=1 per section
    sizes.append((sx, sy))
maxX = max(sx for sx, _ in sizes)
maxY = max(sy for _, sy in sizes)
Z    = len(sections)
print(f"Stack target size per channel: X={maxX}, Y={maxY}, Z={Z}")

# ---- Helper: load & pad a single section/channel to (maxX,maxY) ----
def load_section_ch(section_path: Path, ch: str, maxX: int, maxY: int) -> np.ndarray:
    vol = CloudVolume(cloudpath(section_path / ch), progress=False, bounded=False)
    # read full volume (X,Y,Z). For per-section it’s usually Z=1
    arr = vol[:, :, :].squeeze()  # (X,Y) or (X,Y,Z) -> (X,Y)
    if arr.ndim == 3 and arr.shape[-1] == 1:
        arr = arr[..., 0]
    if arr.ndim != 2:
        raise RuntimeError(f"Unexpected array shape for {section_path.name}/{ch}: {arr.shape}")

    X, Y = arr.shape[0], arr.shape[1]
    out = np.zeros((maxX, maxY), dtype=np.uint16)
    out[:X, :Y] = arr.astype(np.uint16, copy=False)
    return out

# ---- Build one stacked layer per channel ----
for ch in channels:
    print(f"\nChannel: {ch}")
    # Load & pad each section into a list
    planes = []
    for i, s in enumerate(sections, 1):
        print(f"  [{i}/{Z}] {s.name} -> reading {ch} …", end="", flush=True)
        arr = load_section_ch(s, ch, maxX, maxY)  # (X,Y)
        planes.append(arr)
        print(" ok")

    # Stack along Z -> (X,Y,Z)
    stack = np.stack(planes, axis=2).astype(np.uint16, copy=False)
    print(f"  Stacked shape: {stack.shape}")

    # Prepare output layer directory
    out_dir = base_dir / f"{ch}_split"
    out_dir.mkdir(parents=True, exist_ok=True)

    # Create precomputed metadata and write
    info = CloudVolume.create_new_info(
        num_channels=1,
        layer_type="image",
        data_type=dtype,
        encoding="raw",
        resolution=[1, 1, 1],   # set real nm if you know them
        voxel_offset=[0, 0, 0],
        chunk_size=[chunk_xy, chunk_xy, 1],
        volume_size=[int(maxX), int(maxY), int(Z)],
    )
    cp = cloudpath(out_dir)
    print("  Writing to:", cp)
    vol_out = CloudVolume(cp, info=info, progress=True, compress=False)
    vol_out.commit_info()
    vol_out.commit_provenance()
    vol_out[:, :, :] = stack  # writes all chunks
    print(f"  ✅ Wrote {ch} to {out_dir}")

print("\nDone. Open each *_split folder in Neuroglancer as separate layers.")


Found 14 sections: ['section_00', 'section_01', 'section_02', 'section_03', 'section_04'] ...


InfoUnavailableError: [1;91mNo info file was found: file://C:\D\cjDove-DAE8-Systemic\cjDove_confocal_neuroglancer_per_section\section_00\DAPI\info[m

# To use the link below, serve the following data in command prompt

cd to directory with confocal full stack per section data:

cd /d D:\BCH-cjDAE8-mHGHpA\cjSwan\confocal_neuroglancer_dataset_per_section

or cd to directory with epi full stack per section data

cd /d C:\D\BCH-cjDAE8-mHGHpA\cjSwan\neuroglancer_dataset_per_section

http-server . -p 8080 --cors

# Link for composite formatted
    
https://neuroglancer-demo.appspot.com/#!{
  "layers": {
    "DAPI": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/DAPI_split/",
      "shader": "#uicontrol invlerp normalized\nvoid main() {\n  emitRGB(vec3(0, 0, normalized()));\n}",
      "blend": "additive",
      "opacity": 1.0
    },
    "GFP": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/GFP_split/",
      "shader": "#uicontrol invlerp normalized\nvoid main() {\n  emitRGB(vec3(0, normalized(), 0));\n}",
      "blend": "additive",
      "opacity": 1.0
    },
    "TXRED": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/TXRED_split/",
      "shader": "#uicontrol invlerp normalized\nvoid main() {\n  emitRGB(vec3(normalized(), 0, 0));\n}",
      "blend": "additive",
      "opacity": 1.0
    },
    "CY5": {
      "type": "image",
      "source": "precomputed://http://127.0.0.1:8080/CY5_split/",
      "shader": "#uicontrol invlerp normalized\nvoid main() {\n  emitRGB(vec3(normalized(), 0, normalized()));\n}",
      "blend": "additive",
      "opacity": 1.0
    }
  }
}
